In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 3, Finished, Available)

In [2]:
url = 'https://www.lequipe.fr/Rugby/RugbyFicheClub4000000000000170.html'

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 4, Finished, Available)

In [3]:
response = requests.get(url, verify=False)

#Code bonne reponse
if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')

    # Trouver toutes les lignes de résultat
    result_rows = soup.select('.Palmares.calendrierclub .data-content.category-expand-list tr:not(:first-child)')

    data = []

    for row in result_rows:
        try :
            date_all = row.select_one('.date').text.strip().split(" ")
            date, hour = date_all[0], date_all[1]
            
            equipe1 = row.select_one('.equipe.equipe1').text.strip()
            equipe2 = row.select_one('.equipe.equipe2').text.strip()
            score = row.select_one('.score').text.strip()
        except : 
            break
        
        data.append({"Date": date, "Hour" : hour, "Equipe_Home": str(equipe1), "Equipe_Away": str(equipe2), "Score": str(score)})
        
else : 
    print("Problème pour accèder au site")

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 5, Finished, Available)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.lequipe.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
df = pd.DataFrame(data).astype(str)
display(df)

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, ab389420-5996-4629-b4bc-4329cc04194e)

In [5]:
# Nettoyer la colonne "Score" en supprimant les espaces insécables et conserver le "-"
df['Score'] = df['Score'].str.replace('\xa0-\xa0', '-').astype(str)

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 7, Finished, Available)

In [6]:
df[['Score_Home', 'Score_Away']] = df['Score'].str.split("-", n=1, expand=True).astype(int)
df = df.drop(columns="Score")
df.head()

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 8, Finished, Available)

,Date,Hour,Equipe_Home,Equipe_Away,Score_Home,Score_Away
0,04/09/22,21:05:00,Bordeaux Bègles,Toulouse,25,26
1,10/09/22,21:05:00,Montpellier,Bordeaux Bègles,29,19
2,17/09/22,15:00:00,Bordeaux Bègles,Castres,33,12
3,24/09/22,17:00:00,Bayonne,Bordeaux Bègles,20,15
4,01/10/22,15:00:00,Bordeaux Bègles,St. Français,15,10


In [7]:
# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Show the Spark DataFrame
spark_df.show(3)

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 9, Finished, Available)

+--------+--------+---------------+---------------+----------+----------+
|    Date|    Hour|    Equipe_Home|    Equipe_Away|Score_Home|Score_Away|
+--------+--------+---------------+---------------+----------+----------+
|04/09/22|21:05:00|Bordeaux Bègles|       Toulouse|        25|        26|
|10/09/22|21:05:00|    Montpellier|Bordeaux Bègles|        29|        19|
|17/09/22|15:00:00|Bordeaux Bègles|        Castres|        33|        12|
+--------+--------+---------------+---------------+----------+----------+
only showing top 3 rows



In [8]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, when, date_format, to_timestamp, split, to_date

StatementMeta(, 21e87319-1965-4cdb-81d3-7d034f8b1339, 10, Finished, Available)

In [9]:
spark_df = spark_df.withColumn("Ecart_score", col("Score_Home").cast("int") - col("Score_Away").cast("int"))
spark_df.show()

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 11, Finished, Available)

+--------+--------+---------------+---------------+----------+----------+-----------+
|    Date|    Hour|    Equipe_Home|    Equipe_Away|Score_Home|Score_Away|Ecart_score|
+--------+--------+---------------+---------------+----------+----------+-----------+
|04/09/22|21:05:00|Bordeaux Bègles|       Toulouse|        25|        26|         -1|
|10/09/22|21:05:00|    Montpellier|Bordeaux Bègles|        29|        19|         10|
|17/09/22|15:00:00|Bordeaux Bègles|        Castres|        33|        12|         21|
|24/09/22|17:00:00|        Bayonne|Bordeaux Bègles|        20|        15|          5|
|01/10/22|15:00:00|Bordeaux Bègles|   St. Français|        15|        10|          5|
|09/10/22|21:05:00|           Lyon|Bordeaux Bègles|        36|        21|         15|
|15/10/22|21:05:00|Bordeaux Bègles|         Racing|        29|        17|         12|
|22/10/22|21:05:00|       Clermont|Bordeaux Bègles|        23|        23|          0|
|30/10/22|21:05:00|Bordeaux Bègles|         Toulon|   

In [10]:
#spark_df = spark_df.withColumn("date", spark_df["date"].cast("timestamp"))

spark_df.printSchema()


StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 12, Finished, Available)

root
 |-- Date: string (nullable = true)
 |-- Hour: string (nullable = true)
 |-- Equipe_Home: string (nullable = true)
 |-- Equipe_Away: string (nullable = true)
 |-- Score_Home: long (nullable = true)
 |-- Score_Away: long (nullable = true)
 |-- Ecart_score: integer (nullable = true)



In [11]:
regex_ubb = "%Bordeaux%"



result_df = spark_df.withColumn(
        "Result",
        when((col("Equipe_Home").like(regex_ubb)) & (col("Ecart_score") > 0), "Win")
        .when((col("Equipe_Home").like(regex_ubb)) & (col("Ecart_score") == 0), "Draw")
        .when((col("Equipe_Home").like(regex_ubb)) & (col("Ecart_score") < 0), "Loose")
        .when((col("Equipe_Away").like(regex_ubb)) & (col("Ecart_score") < 0), "Win")
        .when((col("Equipe_Away").like(regex_ubb)) & (col("Ecart_score") == 0), "Draw")
        .when((col("Equipe_Away").like(regex_ubb)) & (col("Ecart_score") > 0), "Loose")
    )
    

display(result_df.tail(4))

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, af4b7fbe-6776-4cbe-be06-a9ba82856272)

In [12]:
delta_table_name = "Ubb_results"

result_df.write.mode("overwrite").option("mergeSchema", "false").format("delta").saveAsTable(delta_table_name)

StatementMeta(, 2ca392f3-3cee-4d42-ace5-45a0b7708334, 14, Finished, Available)

JOIN

In [2]:
CLUB_TOP14 = spark.sql("SELECT * FROM Lakehouse.CLUB_TOP14 LIMIT 1000")
display(CLUB_TOP14)

StatementMeta(, 21e87319-1965-4cdb-81d3-7d034f8b1339, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8fb33fd0-9807-4e76-a2b8-49e8fda80d86)

In [3]:
ubb_results = spark.sql("SELECT * FROM Lakehouse.ubb_results LIMIT 10")
display(ubb_results)

StatementMeta(, 21e87319-1965-4cdb-81d3-7d034f8b1339, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 89a252f4-35ff-48fb-ad6f-92d0f378f86a)

In [4]:
final_table = spark.sql(""" WITH table1 as (
SELECT *,
  CASE 
    WHEN ubb_results.Equipe_Away LIKE '%Bordeaux%'
    THEN ubb_results.Equipe_Home
    ELSE ubb_results.Equipe_Away
  END AS Adversaire
FROM Lakehouse.ubb_results
),

finaltable as (SELECT a.*, coalesce(b.Pays, 'Etranger') as Pays_adversaire
FROM table1 as a
LEFT JOIN Lakehouse.CLUB_TOP14 as b
ON a.Adversaire = b.Club
)

SELECT * from finaltable
"""
)

display(final_table.head(3))

StatementMeta(, 21e87319-1965-4cdb-81d3-7d034f8b1339, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 38932fea-4823-4714-9f8b-d4883129dc8d)

In [9]:
# final_table = final_table.withColumn("Date", to_date("Date"))
# display(final_table)

StatementMeta(, 21e87319-1965-4cdb-81d3-7d034f8b1339, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4b7b16a0-3789-47ac-b779-b0d20b2130e2)

In [5]:
final_table.write.mode("overwrite").option("mergeSchema", "false").format("delta").saveAsTable("Table_for_BI")

StatementMeta(, 21e87319-1965-4cdb-81d3-7d034f8b1339, 7, Finished, Available)